In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import shutil
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/aerial-cactus-identification/train.csv
/kaggle/input/aerial-cactus-identification/train.zip
/kaggle/input/aerial-cactus-identification/sample_submission.csv
/kaggle/input/aerial-cactus-identification/test.zip


In [2]:
import zipfile
with zipfile.ZipFile("/kaggle/input/aerial-cactus-identification/train.zip","r") as zip_ref:
    zip_ref.extractall("./")

In [3]:
with zipfile.ZipFile("/kaggle/input/aerial-cactus-identification/test.zip","r") as zip_ref:
    zip_ref.extractall("./")

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
import cv2
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, Dataset
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [5]:
df=pd.read_csv("/kaggle/input/aerial-cactus-identification/train.csv")
df.head()

,id,has_cactus
0,0004be2cfeaba1c0361d39e2b000257b.jpg,1
1,000c8a36845c0208e833c79c1bffedd1.jpg,1
2,000d1e9a533f62e55c289303b072733d.jpg,1
3,0011485b40695e9138e92d0b3fb55128.jpg,1
4,0014d7a11e90b62848904c1418fc8cf2.jpg,1


In [6]:
df.describe()

,has_cactus
count,17500.000000
mean,0.750629
std,0.432662
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,1.000000


In [7]:
tdf=pd.read_csv("/kaggle/input/aerial-cactus-identification/sample_submission.csv")
tdf.head()

,id,has_cactus
0,000940378805c44108d287872b2f04ce.jpg,0.5
1,0017242f54ececa4512b4d7937d1e21e.jpg,0.5
2,001ee6d8564003107853118ab87df407.jpg,0.5
3,002e175c3c1e060769475f52182583d0.jpg,0.5
4,0036e44a7e8f7218e9bc7bf8137e4943.jpg,0.5


In [8]:
df.describe()

,has_cactus
count,17500.000000
mean,0.750629
std,0.432662
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,1.000000


In [9]:
transform=transforms.Compose([transforms.ToPILImage(),transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [10]:
num_epochs=25
num_classes=2
batch_size=128
learning_rate=0.001


In [11]:
train,val=train_test_split(df,test_size=0.1)

In [12]:
train.shape,val.shape

((15750, 2), (1750, 2))

In [13]:
class mydata(Dataset):
    def __init__(self,dfdata,data_dir,transform=None):
        super().__init__()
        self.df=dfdata.values
        self.data_dir=data_dir
        self.transform=transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,index):
        img_name,label=self.df[index]
        img_path=os.path.join(self.data_dir,img_name)
        image=cv2.imread(img_path)
        if self.transform is not None:
            image=self.transform(image)
        return image,label


        
    

In [14]:
train_data=mydata(train,"./train/",transform)
test_data=mydata(val,"./train/",transform)

train_loader=DataLoader(dataset=train_data,batch_size=batch_size,shuffle=True,num_workers=2)
test_loader=DataLoader(dataset=test_data,batch_size=batch_size//2,shuffle=False,num_workers=2)


In [15]:
test_data[1]

(tensor([[[-0.3176, -0.1216, -0.0980,  ...,  0.2706,  0.2000,  0.1922],
          [-0.1608,  0.0039,  0.0667,  ...,  0.2078,  0.2235,  0.3098],
          [-0.2627, -0.1137,  0.1059,  ...,  0.2000,  0.2078,  0.3020],
          ...,
          [ 0.0353,  0.0667, -0.0118,  ..., -0.2392, -0.3020, -0.2471],
          [-0.0353, -0.0039, -0.0588,  ..., -0.2392, -0.2157, -0.2863],
          [-0.0118,  0.0118, -0.0667,  ..., -0.1922, -0.0118, -0.1529]],
 
         [[-0.2235, -0.0275, -0.0039,  ...,  0.2549,  0.1686,  0.1608],
          [-0.0667,  0.0980,  0.1608,  ...,  0.1922,  0.1922,  0.2784],
          [-0.1686, -0.0196,  0.2000,  ...,  0.1843,  0.1922,  0.2863],
          ...,
          [ 0.1765,  0.2078,  0.1294,  ..., -0.0980, -0.1608, -0.1059],
          [ 0.1059,  0.1373,  0.0824,  ..., -0.0980, -0.0745, -0.1451],
          [ 0.1294,  0.1529,  0.0745,  ..., -0.0510,  0.1294, -0.0118]],
 
         [[-0.1294,  0.0667,  0.0902,  ...,  0.3490,  0.2627,  0.2549],
          [ 0.0275,  0.1922,

In [16]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        
        self.pool = nn.MaxPool2d(2, 2)
        self.conv1=nn.Conv2d(3,32,2,padding=2)
        self.bn1=nn.BatchNorm2d(32)
        self.conv2=nn.Conv2d(32,64,2,padding=2)
        self.bn2=nn.BatchNorm2d(64)
        self.conv3=nn.Conv2d(64,128,2,padding=2)
        self.bn3=nn.BatchNorm2d(128)
        self.conv4=nn.Conv2d(128,256,2,padding=2)
        self.bn4=nn.BatchNorm2d(256)
        self.conv5=nn.Conv2d(256,512,2,padding=2)
        self.bn5=nn.BatchNorm2d(512)
        self.fc1=nn.Linear(512*3*3,2)
        
    def forward(self,x):
        x=self.pool(F.relu(self.bn1(self.conv1(x))))
        x=self.pool(F.relu(self.bn2(self.conv2(x))))
        x=self.pool(F.relu(self.bn3(self.conv3(x))))
        x=self.pool(F.relu(self.bn4(self.conv4(x))))
        x=self.pool(F.relu(self.bn5(self.conv5(x))))
#         print(x.shape)
        x=x.view(-1,512*3*3)
        x=self.fc1(x)
        return x
    
        

In [17]:
net=Net().to(device)

In [18]:
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(net.parameters(),lr=learning_rate)

In [19]:
total_step=len(train_loader)
for epoch in range(num_epochs):
    for i,(images,labels) in enumerate(train_loader):
        images=images.to(device)
        labels=labels.to(device)
        
        outputs=net(images)
        loss=criterion(outputs,labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1)%100==0:
            print('Epoch:[{}/{}] step:[{}/{}] loss: {:.4f}'.format(epoch+1,num_epochs,i+1,total_step,loss.item()))

Epoch:[1/25] step:[100/124] loss: 0.1228
Epoch:[2/25] step:[100/124] loss: 0.0446
Epoch:[3/25] step:[100/124] loss: 0.0224
Epoch:[4/25] step:[100/124] loss: 0.0339
Epoch:[5/25] step:[100/124] loss: 0.0163
Epoch:[6/25] step:[100/124] loss: 0.0412
Epoch:[7/25] step:[100/124] loss: 0.0074
Epoch:[8/25] step:[100/124] loss: 0.0035
Epoch:[9/25] step:[100/124] loss: 0.0551
Epoch:[10/25] step:[100/124] loss: 0.0015
Epoch:[11/25] step:[100/124] loss: 0.0489
Epoch:[12/25] step:[100/124] loss: 0.0038
Epoch:[13/25] step:[100/124] loss: 0.0034
Epoch:[14/25] step:[100/124] loss: 0.0011
Epoch:[15/25] step:[100/124] loss: 0.0004
Epoch:[16/25] step:[100/124] loss: 0.0003
Epoch:[17/25] step:[100/124] loss: 0.0011
Epoch:[18/25] step:[100/124] loss: 0.0011
Epoch:[19/25] step:[100/124] loss: 0.0597
Epoch:[20/25] step:[100/124] loss: 0.0088
Epoch:[21/25] step:[100/124] loss: 0.0019
Epoch:[22/25] step:[100/124] loss: 0.0076
Epoch:[23/25] step:[100/124] loss: 0.0000
Epoch:[24/25] step:[100/124] loss: 0.0000
E

In [20]:
net.eval()
with torch.no_grad():
    correct=0
    total=0
    for images,labels in test_loader:
        images=images.to(device)
        labels=labels.to(device)
        outputs=net(images)
        _,predicted=torch.max(outputs.data,1)
        total+=labels.size(0)
        correct+=(predicted==labels).sum().item()
        
    print("acc= ",100*(correct/total))
    


torch.save(net.state_dict(),'model.ckpt')



acc=  99.54285714285714


In [21]:
sub_dataset=mydata(tdf,'./test/',transform)
sub_loader=DataLoader(dataset=sub_dataset,batch_size=32,shuffle=False,num_workers=0)


In [22]:
net.eval()

preds=[]
for batch_i,(data,target) in enumerate(sub_loader):
    data,target=data.to(device),target.to(device)
    output=net(data)
    pr=output[:,1].detach().cpu().numpy()
    for i in pr:
        preds.append(i)
    

tdf['has_cactus']=preds
tdf.to_csv('submission.csv',index=False)

In [23]:
shutil.rmtree('./test')
shutil.rmtree('./train')